<a href="https://colab.research.google.com/github/Ayanlola2002/Stutern/blob/master/Stutern_Group_Task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd
all_accounts=pd.read_csv('/content/drive/My Drive/raw/all_accounts.csv')
all_balances=pd.read_csv('/content/drive/My Drive/raw/all_balances.csv')
#all_demographics=pd.read_csv('/content/drive/My Drive/raw/all_demographics.csv')
all_investments=pd.read_csv('/content/drive/My Drive/raw/all_investments.csv')
all_loans=pd.read_csv('/content/drive/My Drive/raw/all_loans.csv')
all_loans_application=pd.read_csv('/content/drive/My Drive/raw/all_loans_application.csv')
all_transactions_digital=pd.read_csv('/content/drive/My Drive/raw/all_transactions_digital.csv')
bureau_score=pd.read_csv('/content/drive/My Drive/raw/bureau_score.csv')
all_dictionary=pd.read_csv('/content/drive/My Drive/raw/dictionary.csv')

In [33]:
all_dictionary

,,,,Table 1
FEATURES,DESCRIPTION,DATA TYPE,NaN,NaN
BRN,Branch code,VARCHAR2(3),NaN,NaN
ACC,Account number or GL code,VARCHAR2(20),NaN,NaN
VAL_DT,Value date,DATE,NaN,NaN
BAL,Balance,"NUMBER(24,3)",NaN,NaN
...,...,...,...,...
CREDIT_CARDS,Number of Credit cards,NaN,NaN,NaN
E_COUNTRY,Country belonged to,NaN,NaN,NaN
VST_US_PREV,NaN,NaN,NaN,NaN
DATE_OF_BIRTH,Date of birth,NaN,NaN,NaN


In [59]:
#we use the all_loans_applications customer id 
#this indicates all those customers that apply for loans 
#it may also be possible that a customer ask for multiple loans
print(all_loans.shape)
print(all_loans_application.shape)

(278505, 10)
(23172, 8)


In [0]:
all_loans_application.set_index('CUSTOMER_UNIQUE_ID',inplace=True)

In [0]:
important_indexes=all_loans_application.index.unique()

In [0]:
#resetting of all_accounts index
all_accounts.set_index('CUSTOMER_UNIQUE_ID',inplace=True)


In [63]:
all_accounts=all_accounts.loc[important_indexes]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  """Entry point for launching an IPython kernel.


In [70]:
all_accounts['RECORD_STAT'].value_counts()

O    27281
C     1776
Name: RECORD_STAT, dtype: int64

In [69]:
all_accounts.shape

(29068, 4)

In [0]:
all_accounts.drop(columns='MASKED_ACCOUNT',inplace=True)

In [73]:
#checking different type of currency
all_accounts['CCY'].value_counts()

NGN    26765
USD     2082
GBP      146
EUR       64
Name: CCY, dtype: int64

In [0]:
def con(data):
    data.loc[data.CCY=='USD','ACY_AVL_BAL']=data.loc[data.CCY=='USD','ACY_AVL_BAL'].apply(lambda x: x*365)
    data.loc[data.CCY=='EUR','ACY_AVL_BAL']=data.loc[data.CCY=='EUR','ACY_AVL_BAL'].apply(lambda x: x*398)
    data.loc[data.CCY=='GBP','ACY_AVL_BAL']=data.loc[data.CCY=='GPB','ACY_AVL_BAL'].apply(lambda x: x*467)
    data.loc[data.CCY=='JPY','ACY_AVL_BAL']=data.loc[data.CCY=='JPY','ACY_AVL_BAL'].apply(lambda x: x*3.31)
    return data

In [0]:
#changing all account to naira notes
all_accounts=con(all_accounts)

In [0]:

all_accounts.index=all_accounts.index.astype('int')

In [0]:
all_accounts.sort_index(inplace=True)

In [0]:
all_accounts.loc[(all_accounts.ACY_AVL_BAL!=0) & (all_accounts.RECORD_STAT=="C"),'ACY_AVL_BAL']=0